In [9]:
# def get_path(data_path):
#     y = []
#     for paths, folders, all_filenames in os.walk(data_path):
#         for filename in all_filenames:
#             if ".DS_Store" not in filename:
#                 y.append(os.path.join(paths, filename))
#     return y

# 抓取路徑
## 根目錄
## 欲讀取的字串

In [1]:
def get_path(root_folder_path, candidate_str):
    y = []
    for paths, folders, file_NameSpace in os.walk(root_folder_path):
        for filename in file_NameSpace:
            for i in candidate_str:
                if i in filename:
                    y.append(os.path.join(paths, filename))
                    break
    return y

# Label the data
## x:要讀取的路徑string  
## candidate_str:候選字串list

In [2]:
def label_the_data(path_list, candidate_str):
    y_label = np.zeros(len(candidate_str))
    for i in candidate_str:
        if path_list.find(i) != -1:
            y_label[candidate_str.index(i)] = 1
            break
    return y_label

In [11]:
def array_delete(x):
    # delete columns
    x = np.delete(x, range(26), 0)
    x = np.delete(x, [17, 18], 0)
   
    # delete rows
    x = np.delete(x, 39, 1)
    
    return x

In [12]:
def connect_data_folder(data_path, candiate_str):
    load_path = get_path(data_path, candiate_str)
    p = len(load_path)
    
    sample = np.array(pd.read_csv(load_path[0], header=None))
    sample = array_delete(sample)
    
    nr, nc = sample.shape
    n = nr * nc
    x_input = np.zeros((p, n))
    y_input = np.zeros((p, len(candiate_str)))
    for i in range(p):
        x_trans = np.array(pd.read_csv(load_path[i], header=None))
        x_trans = array_delete(x_trans)
        x_input[i] = x_trans.reshape((1, n))
        y_input[i] = label_the_data(load_path[i], candiate_str)
    return x_input, y_input

In [3]:
# data_tuple : 切割前的可訓練data組
# num_fr : 欲切割的frame數
# 輸出一組可訓練tuple
# length : 單筆資料攤平後的長度，依據所選的音框不同有差別
# raw data : sample_size x length
# 舉例:raw data 516 x 25  切割後: 25 x 516 x 39

# def frame_split(data_tuple, num_fr):
#     sp_frame_x = np.array(np.split(data_tuple[0], num_fr, 1))
#     sp_data_list = list(np.zeros(num_fr))
#     for i in range(num_fr):
#         sp_data_list[i] = sp_frame_x[i, :, :], data_tuple[1]
#     return sp_data_list

In [3]:
#1108 更新版1
def frame_split22(data_tuple, num_fr):
    sp_frame_x = np.array(np.split(data_tuple[0], num_fr, 1))
    return sp_frame_x, data_tuple[1]

In [4]:
#1108 更新版2
# def frame_split(data_tuple, num_fr):
#     sp_frame_x = np.array(np.split(data_tuple[0], num_fr, 1))
#     sp_frame_y = np.zeros([num_fr, data_tuple[1].shape[0], data_tuple[1].shape[1]])
    
#     for i in range(num_fr):
#         sp_frame_y[i, :, :] = data_tuple[1]
#     return sp_frame_x, sp_frame_y

In [ ]:
# 功能：將矩陣放入，輸出(逐行或逐列標準化後的矩陣, 標準化用的mean_matrix, 標準化用的sd_matrix)
# train_x：放入的矩陣
# by_row：bool, 逐列則True, 逐行則false
# def scale_apply(train_x, by_row = True):
#     shape = train_x.shape
#     if by_row == True:
#         mean_vec = train_x.mean(axis=1).reshape([shape[0], 1])
#         var_vec = train_x.var(axis=1).reshape([shape[0], 1])
#         one = np.zeros([1, shape[1]]) + 1
        
#         mean_mat = np.matmul(mean_vec, one)
#         var_mat = np.matmul(var_vec, one)
#         y = (train_x - mean_mat) / var_mat**(0.5)
#     else:
#         mean_vec = train_x.mean(axis=0).reshape([shape[0], 1])
#         var_vec = train_x.var(axis=0).reshape([shape[0], 1])
#         one = np.zeros([1, shape[1]]) + 1
        
#         mean_mat = np.matmul(mean_vec, one)
#         var_mat = np.matmul(var_vec, one)
#         y = (train_x - mean_mat) / var_mat**(0.5)
#     return y, mean_vec, var_vec**(0.5)

In [1]:
# def scale_by_train(test_x, mean_vec, sd_vec):
#     shape = test_x.shape
#     one = np.zeros([1, shape[1]]) + 1
#     mean_mat = np.matmul(mean_vec, one)
#     sd_mat = np.matmul(sd_vec, one)
#     y = (train_x - mean_mat) / sd_mat
    
#     return y

# 標準化

## 抓取train data的scale

In [ ]:
def get_mean_and_sd(train_x):
    tr_frame_x = np.array(np.split(train_x, 25, 1))
    tr_mean = tr_frame_x.mean(axis=1, keepdims=True)
    tr_sd = np.var(tr_frame_x, axis=1, keepdims=True)
    return tr_mean, tr_sd

## 利用抓取好的scale進行標準化

In [ ]:
def scale_by_train(sp_x, tr_mean, tr_sd):
    sample_size = sp_x.shape[1]
    temp = sp_x.copy()
    for i in range(sample_size):
#         temp[:, i, :] = np.add(temp[:, i, :], tr_mean)
#         temp[:, i, :] = np.divide(temp[:, i, :], tr_sd)
        temp[:, i, :] -= tr_mean[:, 0, :]
        temp[:, i, :] /= (tr_sd[:, 0, :])**0.5
    return temp